In [3]:
%pip install fastapi uvicorn nest_asyncio pyngrok


Note: you may need to restart the kernel to use updated packages.


In [4]:
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import uvicorn
import torch

# Apply nest_asyncio
nest_asyncio.apply()

app = FastAPI()

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

class InferenceRequest(BaseModel):
    prompt: str
    max_length: int = 50
    num_return_sequences: int = 1

@app.post("/gpt2-inference/")
def gpt2_inference(request: InferenceRequest):
    inputs = tokenizer(request.prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids,
        max_length=request.max_length,
        num_return_sequences=request.num_return_sequences
    )
    responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"responses": responses}

# Start ngrok to tunnel the localhost
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the application
uvicorn.run(app, host="127.0.0.1", port=8000)


Public URL: NgrokTunnel: "https://557c-41-86-177-7.ngrok-free.app" -> "http://localhost:8000"       


INFO:     Started server process [2652]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     41.86.177.7:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     41.86.177.7:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     127.0.0.1:64935 - "POST /gpt2-inference/ HTTP/1.1" 200 OK


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     41.86.177.7:0 - "POST /gpt2-inference/ HTTP/1.1" 200 OK


In [ ]:
%pip install requests